In [4]:
import pandas as pd

In [5]:
train = pd.read_csv('C:/Users/Nikita/OneDrive/Desktop/Course_Materials/NLP/Assignments/Assignment2/offensive-language-detection/data/olid-train.csv')
test = pd.read_csv('C:/Users/Nikita/OneDrive/Desktop/Course_Materials/NLP/Assignments/Assignment2/offensive-language-detection/data/olid-test.csv')
diagnostic = pd.read_csv('C:/Users/Nikita/OneDrive/Desktop/Course_Materials/NLP/Assignments/Assignment2/offensive-language-detection/data/olid-subset-diagnostic-tests.csv')

In [6]:
train.head()

,id,text,labels
0,86426,@USER She should ask a few native Americans wh...,1
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,1
2,16820,Amazon is investigating Chinese employees who ...,0
3,62688,"@USER Someone should'veTaken"" this piece of sh...",1
4,43605,@USER @USER Obama wanted liberals &amp; illega...,0


In [7]:
test.head()

,id,text,labels
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,1
1,27014,"#ConstitutionDay is revered by Conservatives, ...",0
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0
3,13876,#Watching #Boomer getting the news that she is...,0
4,60133,#NoPasaran: Unity demo to oppose the far-right...,1


In [8]:
diagnostic.head()

,id,text,labels
0,89200,@USER @USER Who the hell does he think he is?,1
1,71294,#BREAKING. #Greece: Molotov cocktails fly afte...,1
2,55633,"#OrrinHatch I can’t believe this sexist , clue...",1
3,16856,@USER @USER I'll use that one the next time im...,1
4,26072,0-1 lost my acca on the first fucking fight cba,1


In [10]:
#pip install simpletransformers

     -------------------------------------- 249.1/249.1 KB 7.7 MB/s eta 0:00:00
     ---------------------------------------- 43.6/43.6 KB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.1/1.1 MB 9.8 MB/s eta 0:00:00
     ---------------------------------------- 5.8/5.8 MB 2.6 MB/s eta 0:00:00
     --------------------------------------- 10.1/10.1 MB 10.4 MB/s eta 0:00:00
     ---------------------------------------- 4.0/4.0 MB 11.0 MB/s eta 0:00:00
     ------------------------------------- 342.3/342.3 KB 10.7 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 11.0 MB/s eta 0:00:00
     ---------------------------------------- 1.8/1.8 MB 11.4 MB/s eta 0:00:00
     ------------------------------------- 880.6/880.6 KB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   

In [11]:
#pip show simpletransformers

Name: simpletransformers
Version: 0.63.6
Summary: An easy-to-use wrapper library for the Transformers library.
Home-page: https://github.com/ThilinaRajapakse/simpletransformers/
Author: Thilina Rajapakse
Author-email: chaturangarajapakshe@gmail.com
License: UNKNOWN
Location: c:\users\nikita\anaconda3\lib\site-packages
Requires: datasets, numpy, pandas, regex, requests, scikit-learn, scipy, sentencepiece, seqeval, streamlit, tensorboard, tokenizers, tqdm, transformers, wandb
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [12]:
#pip install --upgrade simpletransformers

Note: you may need to restart the kernel to use updated packages.


In [9]:
from random import randint
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import random
import spacy

In [10]:
# PART A

In [11]:
# 1. Class distributions (1 point)
# Load the training set (olid-train.csv) and analyze the number of instances for each of the two classification labels.

# Number of instances for class '0': 8840
zero = train[(train['labels']== 0)]
num_zero = len(zero)

# Number of instances for class '1': 4400
one = train[(train['labels']== 1)]
num_one = len(one)

In [12]:
print('Number of instances for label 0: ', num_zero)
print('Number of instances for label 1: ', num_one)

Number of instances for label 0:  8840
Number of instances for label 1:  4400


In [13]:
# Relative label frequency (%)
total_num = len(train)
# Frequency of label 0 : 0.6676737160120846 ~ 0.67 = 67%
zero_frequency = num_zero/total_num
# Frequency of label 1 : 0.3323262839879154 ~ 0.33 = 33%
one_frequency = num_one/total_num

In [14]:
print('Relative Frequency for label 0: ', zero_frequency)
print('Relative Frequency for label 1: ', one_frequency)

Relative Frequency for label 0:  0.6676737160120846
Relative Frequency for label 1:  0.3323262839879154


In [21]:
# Example tweet for each label
# Example tweet for label 0: 8 @USER Buy more icecream!!!
label0_text = train['text'][(train['labels']==0)]
# Example tweet for label 1: 5 @USER Liberals are all Kookoo !!!
label1_text = train['text'][(train['labels']==1)]

In [22]:
print('Examples of tweets with label 0 :', label0_text)

Examples of tweets with label 0 : 2        Amazon is investigating Chinese employees who ...
4        @USER @USER Obama wanted liberals &amp; illega...
8                               @USER Buy more icecream!!!
10       @USER @USER @USER It’s not my fault you suppor...
11       @USER What’s the difference between #Kavanaugh...
                               ...                        
13232    @USER She is not the brightest light on the tree.
13233    @USER 😂😂😂😂😂😂 if I say you are mad now you will...
13234    @USER @USER @USER @USER @USER @USER @USER @USE...
13236    Benidorm ✅  Creamfields ✅  Maga ✅   Not too sh...
13239    #Spanishrevenge vs. #justice #HumanRights and ...
Name: text, Length: 8840, dtype: object


In [23]:
print('Examples of tweets with label 1 :', label1_text)

Examples of tweets with label 1 : 0        @USER She should ask a few native Americans wh...
1        @USER @USER Go home you’re drunk!!! @USER #MAG...
3        @USER Someone should'veTaken" this piece of sh...
5                        @USER Liberals are all Kookoo !!!
6                         @USER @USER Oh noes! Tough shit.
                               ...                        
13223    @USER is advocating for conduct within bounds ...
13227    @USER @USER @USER @USER Liars like the Antifa ...
13235    @USER Sometimes I get strong vibes from people...
13237    @USER And why report this garbage.  We don't g...
13238                                          @USER Pussy
Name: text, Length: 4400, dtype: object


In [24]:
# 2. Baselines (1 point)
# Calculate two baselines and evaluate their performance on the test set (olid-test.csv): 
# ● The first baseline is a random baseline that randomly assigns one of the 2 classification labels. 
# ● The second baseline is a majority baseline that always assigns the majority class. 
# Calculate the results on the test set and fill them into the two tables below. Round the results to two decimals. 

In [27]:
# Random baseline
def random_baseline(train_text, test_text, test_labels):
    labels = [0, 1]

    predictions = []
    for instance in test_text:
        instance_prediction = [random.choice(labels)]
        predictions.append(instance_prediction)

    predictions = [item for sublist in predictions for item in sublist]

    test_labels_list = []
    for element in test_labels:
        test_labels_list.append(element)

    prediction_df = pd.DataFrame({"predictions": predictions})

    accuracy = accuracy_score(test_labels_list, predictions)
    print("Accuracy: ", accuracy)

    print(classification_report(test_labels_list, predictions, zero_division=1))

In [28]:
random_baseline(train['text'], test['text'], test['labels'])

Accuracy:  0.48255813953488375
              precision    recall  f1-score   support

           0       0.70      0.49      0.57       620
           1       0.26      0.47      0.34       240

    accuracy                           0.48       860
   macro avg       0.48      0.48      0.46       860
weighted avg       0.58      0.48      0.51       860



In [29]:
def majority_baseline(train_label, test_text, test_label):
    train_label_list = []

    for element in train_label:
        train_label_list.append(element)

    label_one = train_label_list.count(1)
    label_zero = train_label_list.count(0)

    if label_one > label_zero:
        majority_baseline_class = 1
    else:
        majority_baseline_class = 0

    predictions = []
    test_label_list = []
    for text in test_text:
        predictions.append(majority_baseline_class)

    test_label_list = []
    for element in test_label:
        test_label_list.append(element)

    final_df = pd.DataFrame({"predictions": predictions})

    accuracy = accuracy_score(test_label_list, predictions)
    print("Accuracy: ", accuracy)
    print(classification_report(test_label_list, predictions, zero_division=1))


In [30]:
majority_baseline(train['labels'], test['text'], test['labels'])

Accuracy:  0.7209302325581395
              precision    recall  f1-score   support

           0       0.72      1.00      0.84       620
           1       1.00      0.00      0.00       240

    accuracy                           0.72       860
   macro avg       0.86      0.50      0.42       860
weighted avg       0.80      0.72      0.60       860



In [31]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
from sklearn.metrics import classification_report
import sklearn

In [34]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.learning_rate = 1e-4

model = ClassificationModel("bert", "bert-base-cased", args=model_args, use_cuda=False)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [ ]:
# Training model
model.train_model(train)